# Shared State Middleware

This notebook demonstrates how middleware can share state and communicate with each other.

## Key Concepts

- **Context Sharing**: Pass data between middleware
- **State Management**: Maintain shared state across invocations
- **Coordination**: Multiple middleware working together

## Prerequisites

- Azure OpenAI endpoint and deployment configured in `.env`
- `agent-framework` package installed
- Azure CLI authentication

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv('../.env')

project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
model_deployment_name = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME")

print(f"Project Endpoint: {project_endpoint}")
print(f"Model Deployment: {model_deployment_name}")

In [ ]:
from collections.abc import Awaitable, Callable
from random import randint
from typing import Annotated

from agent_framework import FunctionInvocationContext
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
from pydantic import Field

## Define Tool Functions

In [ ]:
def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """Get the weather for a given location."""
    conditions = ["sunny", "cloudy", "rainy", "stormy"]
    return f"The weather in {location} is {conditions[randint(0, 3)]} with a high of {randint(10, 30)}°C."


def get_time(
    timezone: Annotated[str, Field(description="The timezone to get the time for.")] = "UTC",
) -> str:
    """Get the current time for a given timezone."""
    import datetime
    
    return f"The current time in {timezone} is {datetime.datetime.now().strftime('%H:%M:%S')}"

## Middleware Container with Shared State

This class demonstrates:
- Holding multiple middleware functions in a single class
- Sharing state (call_count) between middleware functions
- First middleware counts calls, second middleware uses the count to enhance results

In [ ]:
class MiddlewareContainer:
    """Container class that holds middleware functions with shared state."""

    def __init__(self) -> None:
        # Simple shared state: count function calls
        self.call_count: int = 0

    async def call_counter_middleware(
        self,
        context: FunctionInvocationContext,
        next: Callable[[FunctionInvocationContext], Awaitable[None]],
    ) -> None:
        """First middleware: increments call count in shared state."""
        # Increment the shared call count
        self.call_count += 1

        print(f"[CallCounter] This is function call #{self.call_count}")

        # Call the next middleware/function
        await next(context)

    async def result_enhancer_middleware(
        self,
        context: FunctionInvocationContext,
        next: Callable[[FunctionInvocationContext], Awaitable[None]],
    ) -> None:
        """Second middleware: uses shared call count to enhance function results."""
        print(f"[ResultEnhancer] Current total calls so far: {self.call_count}")

        # Call the next middleware/function
        await next(context)

        # After function execution, enhance the result using shared state
        if context.result:
            enhanced_result = f"[Call #{self.call_count}] {context.result}"
            context.result = enhanced_result
            print("[ResultEnhancer] Enhanced result with call number")

## Example: Shared State Function-based Middleware

In [ ]:
async def main():
    """Example demonstrating shared state function-based middleware."""
    print("=== Shared State Function-based Middleware Example ===")

    # Create middleware container with shared state
    middleware_container = MiddlewareContainer()
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential).create_agent(
            name="UtilityAgent",
            instructions="You are a helpful assistant that can provide weather information and current time.",
            tools=[get_weather, get_time],
            # Pass both middleware functions from the same container instance
            # Order matters: counter runs first to increment count,
            # then result enhancer uses the updated count
            middleware=[
                middleware_container.call_counter_middleware,
                middleware_container.result_enhancer_middleware,
            ],
        ) as agent,
    ):
        # Test multiple requests to see shared state in action
        queries = [
            "What's the weather like in New York?",
            "What time is it in London?",
            "What's the weather in Tokyo?",
        ]

        for i, query in enumerate(queries, 1):
            print(f"\n--- Query {i} ---")
            print(f"User: {query}")
            result = await agent.run(query)
            print(f"Agent: {result.text if result.text else 'No response'}")

        # Display final statistics
        print("\n=== Final Statistics ===")
        print(f"Total function calls made: {middleware_container.call_count}")

await main()

## Key Takeaways

- Function-based middleware can share state through a container class
- Multiple middleware functions are instance methods of the same class
- Shared state is stored as class instance variables (e.g., `self.call_count`)
- Middleware execution order matters when working with shared state
- First middleware increments count, second middleware uses it
- Useful for metrics collection, coordination, and stateful operations

## Summary

You've now learned all the middleware patterns:
1. **Agent vs Run level** - Apply middleware globally or per-run
2. **Function-based** - Simple async functions for middleware
3. **Class-based** - AgentMiddleware and FunctionMiddleware classes
4. **Decorator** - @agent_middleware and @function_middleware decorators
5. **Chat** - ChatMiddleware for message-level interception
6. **Exception handling** - Catch and handle errors gracefully
7. **Termination** - Early exit with context.terminate
8. **Result override** - Modify results for streaming and non-streaming
9. **Shared state** - Container class with multiple middleware functions

These patterns can be combined to build powerful and flexible agent systems!